In [ ]:
import yfinance as yf
import pandas as pd

nifty50_tickers = [
    "RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS",
    "HINDUNILVR.NS", "ITC.NS", "LT.NS", "SBIN.NS", "KOTAKBANK.NS",
    "AXISBANK.NS", "ASIANPAINT.NS", "BAJFINANCE.NS", "BHARTIARTL.NS",
    "HCLTECH.NS", "SUNPHARMA.NS", "MARUTI.NS", "NTPC.NS", "TITAN.NS",
    "ULTRACEMCO.NS", "WIPRO.NS", "TECHM.NS", "POWERGRID.NS", "NESTLEIND.NS",
    "JSWSTEEL.NS", "TATASTEEL.NS", "ADANIENT.NS", "COALINDIA.NS", "BAJAJFINSV.NS",
    "DIVISLAB.NS", "GRASIM.NS", "HDFCLIFE.NS", "CIPLA.NS", "SBILIFE.NS",
    "BRITANNIA.NS", "EICHERMOT.NS", "HEROMOTOCO.NS", "BAJAJ-AUTO.NS", "INDUSINDBK.NS",
    "BPCL.NS", "SHREECEM.NS", "DRREDDY.NS", "HINDALCO.NS", "ONGC.NS",
    "TATAMOTORS.NS", "APOLLOHOSP.NS", "M&M.NS", "UPL.NS", "ADANIPORTS.NS"
]

data = yf.download(nifty50_tickers, start="2020-01-01", end="2025-01-01")
data.to_csv("nifty50_data.csv")

In [ ]:
# Remove NaN values
data.dropna(inplace=True)

In [ ]:
import numpy as np

In [ ]:
# Show cleaned data

print("Cleaned Data:")
print(data.head())

# Calculate daily returns
returns = data['Close'].pct_change().dropna()

# Calculate log returns
log_returns = np.log(1 + returns).dropna()

print("\nDaily Returns:")
print(returns.head())
print("\nLog Returns:")
print(log_returns.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Summary statistics
print(returns.describe())

# Heatmap of correlations
plt.figure(figsize=(10, 6))
sns.heatmap(returns.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix of Nifty 50 Stocks")
plt.show()

In [ ]:
#Risk return summary

risk_return_summary = pd.DataFrame({
    'Mean Daily Return': returns.mean(),
    'Daily Volatility (Std Dev)': returns.std()
}).sort_values(by='Mean Daily Return', ascending=False)

print("\nRisk-Return Summary:")
print(risk_return_summary)

plt.figure(figsize=(10, 6))
sns.scatterplot(x='Daily Volatility (Std Dev)', y='Mean Daily Return', data=risk_return_summary, s=100)
for ticker in risk_return_summary.index:
    plt.text(risk_return_summary.loc[ticker, 'Daily Volatility (Std Dev)'] + 0.0001,
             risk_return_summary.loc[ticker, 'Mean Daily Return'],
             ticker, fontsize=9)
plt.xlabel('Daily Volatility (Std Dev)')
plt.ylabel('Mean Daily Return')
plt.title('Risk-Return Scatter Plot')
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns
cov_matrix = returns.cov()
corr_matrix = returns.corr()

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
sns.heatmap(cov_matrix, cmap='Blues', center=0)
plt.title('Covariance Matrix')

plt.subplot(1, 2, 2)
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
def predict_returns(returns, n_lags):
    X_all = []
    y_all_dict = {ticker: [] for ticker in returns.columns}

    for i in range(n_lags, len(returns) - 1):
        lagged = returns.iloc[i - n_lags:i].values.flatten()
        X_all.append(lagged)
        for ticker in returns.columns:
            y_all_dict[ticker].append(returns.iloc[i + 1][ticker])

    X = np.array(X_all)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    latest_input = returns.iloc[-n_lags:].values.flatten().reshape(1, -1)
    latest_input_scaled = scaler.transform(latest_input)

    predicted_returns = []
    for ticker in returns.columns:
        y = np.array(y_all_dict[ticker])
        model = RandomForestRegressor(n_estimators=50, random_state=42)
        model.fit(X_scaled[:len(y)], y)
        pred = model.predict(latest_input_scaled)[0]
        predicted_returns.append(pred)
    return np.array(predicted_returns)

In [ ]:
def optimize_portfolio(mu, Sigma, rf, tickers, max_variance=0.0002):
    n = len(mu)

    # MVO (Maximize Return under risk constraint)
    mu_hist = np.mean(mu)
    shrinkage_factor = 0.5
    mu_shrunk = shrinkage_factor * mu + (1 - shrinkage_factor) * mu_hist
    w_mvo = cp.Variable(n)
    portfolio_return = mu_shrunk @ w_mvo
    portfolio_variance = cp.quad_form(w_mvo, Sigma)
    max_weight = 0.3
    constraints = [
    cp.sum(w_mvo) == 1,      # weights sum to 1
    w_mvo >= 0,              # no short selling
    w_mvo <= max_weight,     # max weight constraint
    portfolio_variance <= max_variance  # risk constraint
]
    prob = cp.Problem(cp.Maximize(portfolio_return), constraints)
    prob.solve()
    weights_mvo = w_mvo.value

    # Max Sharpe
    def neg_sharpe(w):
        ret = np.dot(w, mu)
        vol = np.sqrt(np.dot(w.T, np.dot(Sigma, w)))
        return -(ret - rf) / vol

    bounds = [(0, 0.2)] * n
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    init_guess = np.repeat(1/n, n)

    result = minimize(neg_sharpe, init_guess, method='SLSQP',
                      bounds=bounds, constraints=constraints)
    weights_sharpe = result.x

    ret = np.dot(weights_sharpe, mu)
    vol = np.sqrt(np.dot(weights_sharpe.T, np.dot(Sigma, weights_sharpe)))
    sharpe = (ret - rf) / vol

    return weights_mvo, weights_sharpe, ret, vol, sharpe

In [ ]:
def equal_weight_portfolio(mu, Sigma, rf):
    n = len(mu)
    w_eq = np.repeat(1/n, n)
    ret = np.dot(w_eq, mu)
    vol = np.sqrt(np.dot(w_eq.T, np.dot(Sigma, w_eq)))
    sharpe = (ret - rf) / vol
    return w_eq, ret, vol, sharpe

In [ ]:
def _nearest_psd(A, eps=1e-10):
    B = 0.5*(A + A.T)
    w, V = np.linalg.eigh(B)
    w_clipped = np.clip(w, eps, None)
    return (V * w_clipped) @ V.T

def market_implied_delta(returns, rf, market_weights):
    mu_mkt = returns.mean().values @ market_weights
    var_mkt = market_weights.T @ returns.cov().values @ market_weights
    delta = (mu_mkt - rf) / max(var_mkt, 1e-12)   # avoid divide-by-zero
    return float(max(delta, 0.0))

def black_litterman(mu_view, Sigma, rf, tickers, returns, tau=0.2, omega_scalar=0.1):
    n = len(mu_view)
    caps = []
    for tk in tickers:
        try:
            info = yf.Ticker(tk).info
            caps.append(info.get("marketCap", 0))
        except Exception:
            caps.append(0)
    caps = np.array(caps, dtype=float)
    if np.nansum(caps) <= 0:
        market_weights = np.full(n, 1.0/n)
    else:
        market_weights = caps/np.nansum(caps)
    mu_view = np.asarray(mu_view, dtype=float).reshape(-1)
    Sigma = np.asarray(Sigma, dtype=float)
    Sigma_psd = _nearest_psd(Sigma)

    # --- market-implied equilibrium returns ---
    delta = market_implied_delta(returns, rf, market_weights)  # >= 0 by clamp
    Pi = delta * (Sigma_psd @ market_weights)

    # --- Black–Litterman posterior mean ---
    P = np.eye(n)
    Omega = np.eye(n) * omega_scalar
    A = np.linalg.inv(tau * Sigma_psd)
    post_prec = A + P.T @ np.linalg.inv(Omega) @ P
    post_mean = np.linalg.inv(post_prec) @ (A @ Pi + P.T @ np.linalg.inv(Omega) @ mu_view)

    # --- mean-variance portfolio
    w = cp.Variable(n)
    ret = post_mean @ w
    risk = cp.quad_form(w, Sigma_psd)
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(cp.Maximize(ret - delta * risk), constraints)
    prob.solve(solver=cp.SCS, verbose=False)

    if prob.status not in ("optimal", "optimal_inaccurate"):
        target = float(np.dot(post_mean, market_weights))  # market-implied return
        w2 = cp.Variable(n)
        risk2 = cp.quad_form(w2, Sigma_psd)
        cons2 = [cp.sum(w2) == 1, w2 >= 0, post_mean @ w2 >= target]
        prob2 = cp.Problem(cp.Minimize(risk2), cons2)
        prob2.solve(solver=cp.SCS, verbose=False)
        if prob2.status not in ("optimal", "optimal_inaccurate"):
            raise ValueError(f"BL optimization failed: {prob.status}, fallback: {prob2.status}")
        return post_mean, w2.value

    return post_mean, w.value

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Ensure returns is a DataFrame
if isinstance(returns, pd.Series):
    returns = returns.to_frame()

# Replace inf values with NaN
returns.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows (or columns) with NaN values
returns.dropna(inplace=True)  # or use returns.fillna(0) based on your case

# Now apply Z-score normalization
zscore_returns = pd.DataFrame(StandardScaler().fit_transform(returns),
                              columns=returns.columns, index=returns.index)


In [ ]:
import statsmodels.api as sm
import yfinance as yf
import pandas as pd

# Assuming NIFTY index as market return
nifty_index_data = yf.download("^NSEI", start="2020-01-01", end="2025-01-01", auto_adjust=False)
nifty_index = nifty_index_data['Adj Close']
market_returns = nifty_index.pct_change().dropna()
market_returns = market_returns.reindex(returns.index).fillna(method='ffill')

capm_results = {}
for stock in returns.columns:
    X = sm.add_constant(market_returns)
    y = returns[stock]
    model = sm.OLS(y, X).fit()
    capm_results[stock] = model.params

capm_df = pd.DataFrame(capm_results).T
capm_df.columns = ['Alpha', 'Beta']
print(capm_df)

In [ ]:
# Remove rows with NaN values from the returns DataFrame
returns.dropna(inplace=True)

In [ ]:
# Run Black-Litterman Model
# Define risk-free rate (replace with appropriate value)
rf = 0.0  # Assuming a risk-free rate of 0 for daily returns

# Calculate mean returns and covariance matrix from the returns data
mu = returns.mean()
Sigma = returns.cov()

# Using historical mean as a placeholder for mu_view
mu_view = mu.values

import cvxpy as cp

try:
    mu_bl, weights_bl = black_litterman(mu_view, Sigma, rf, nifty50_tickers, returns)

    # Create a DataFrame for Black-Litterman Weights and Posterior Returns
    df_bl = pd.DataFrame({
        'Ticker': nifty50_tickers,
        'Weight (Black-Litterman)': weights_bl,
        'Posterior Return (BL)': mu_bl
    })

    print("\nBlack-Litterman Weights and Posterior Expected Returns:")
    print(df_bl.to_string(index=False))

except ValueError as e:
    print(f"Black-Litterman optimization failed: {e}")
    # Handle the case where BL optimization fails

In [ ]:
# Run Equal-Weight Portfolio Model
# Calculate mean returns and covariance matrix from the returns data
mu = returns.mean()
Sigma = returns.cov()

# Define risk-free rate (replace with appropriate value)
rf = 0.0 # Assuming a risk-free rate of 0 for daily returns

# Calculate Equal-Weight Portfolio
w_eq, ret_eq, vol_eq, sharpe_eq = equal_weight_portfolio(mu, Sigma, rf)

# Create a DataFrame for Equal-Weight Portfolio Weights
df_eq = pd.DataFrame({
    'Ticker': nifty50_tickers,
    'Weight (Equal-Weight)': w_eq
})

# Create a DataFrame for Equal-Weight Portfolio Summary
df_eq_summary = pd.DataFrame({
    'Metric': ['Return', 'Volatility', 'Sharpe Ratio'],
    'Value': [ret_eq, vol_eq, sharpe_eq]
})

print("\nEqual-Weight Portfolio Weights:")
print(df_eq.to_string(index=False))

print("\nEqual-Weight Portfolio Summary:")
print(df_eq_summary.to_string(index=False))

In [ ]:
# Compare CAPM, Black-Litterman, and Equal-Weight Models

# Collect relevant metrics
comparison_metrics = {}

# Get CAPM Alpha and Beta (average across stocks) if available
if 'capm_df' in locals() and not capm_df.empty:
    comparison_metrics['CAPM'] = {
        'Average Alpha': capm_df['Alpha'].mean(),
        'Average Beta': capm_df['Beta'].mean(),
        'Sharpe Ratio': np.nan # CAPM itself doesn't provide a portfolio Sharpe Ratio directly
    }

# Get Black-Litterman Sharpe Ratio if available
if 'weights_bl' in locals() and 'mu_bl' in locals() and 'Sigma' in locals():
    try:
        # Calculate Sharpe Ratio for Black-Litterman portfolio
        ret_bl = np.dot(weights_bl, mu_bl) * 252
        Sigma_np = Sigma.values if isinstance(Sigma, pd.DataFrame) else Sigma
        vol_bl = np.sqrt(np.dot(weights_bl.T, np.dot(Sigma_np, weights_bl))) * np.sqrt(252)
        sharpe_bl = (ret_bl - rf) / (vol_bl + 1e-9) # Add epsilon to avoid division by zero
        if 'Black-Litterman' in comparison_metrics:
            comparison_metrics['Black-Litterman']['Sharpe Ratio'] = sharpe_bl
        else:
            comparison_metrics['Black-Litterman'] = {'Average Alpha': np.nan, 'Average Beta': np.nan, 'Sharpe Ratio': sharpe_bl}
    except Exception as e:
        print(f"Error calculating Sharpe Ratio for Black-Litterman for comparison: {e}")


# Get Equal-Weight Sharpe Ratio if available
if 'df_eq_summary' in locals() and not df_eq_summary.empty:
    eq_sharpe = df_eq_summary[df_eq_summary['Metric'] == 'Sharpe Ratio']['Value'].iloc[0]
    if 'Equal-Weight' in comparison_metrics:
        comparison_metrics['Equal-Weight']['Sharpe Ratio'] = eq_sharpe
    else:
         comparison_metrics['Equal-Weight'] = {'Average Alpha': np.nan, 'Average Beta': np.nan, 'Sharpe Ratio': eq_sharpe}


# Create a DataFrame for comparison
# Create a DataFrame for comparison
comparison_df = pd.DataFrame.from_dict(comparison_metrics, orient='index')

# Clean up NaN values for better display
comparison_df = comparison_df.fillna('')   # replace NaN with blank

print("\nModel Comparison:")
print(comparison_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'comparison_metrics' dictionary is available from the previous cell's execution
if 'comparison_metrics' in locals():
    # Extract Sharpe Ratios for the three portfolio models
    sharpe_ratios = {
        'CAPM': comparison_metrics.get('CAPM', {}).get('Sharpe Ratio'),
        'Black-Litterman': comparison_metrics.get('Black-Litterman', {}).get('Sharpe Ratio'),
        'Equal-Weight': comparison_metrics.get('Equal-Weight', {}).get('Sharpe Ratio')
    }

    # Remove models with no calculated Sharpe Ratio (NaN)
    sharpe_ratios_cleaned = {k: v for k, v in sharpe_ratios.items() if pd.notna(v)}

    if sharpe_ratios_cleaned:
        models = list(sharpe_ratios_cleaned.keys())
        sharpes = list(sharpe_ratios_cleaned.values())

        # Create a bar plot to compare Sharpe Ratios
        plt.figure(figsize=(8, 5))
        bars = plt.bar(models, sharpes, color=['skyblue', 'lightgreen', 'salmon'])
        plt.ylabel('Sharpe Ratio')
        plt.title('Sharpe Ratio Comparison of Portfolio Models')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Add the values on top of the bars
        for bar in bars:
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), va='bottom', ha='center') # Adjust va and ha for positioning

        plt.show()
    else:
        print("No valid Sharpe Ratios available for plotting the comparison.")

else:
    print("Model comparison metrics not available. Please run the cell to calculate comparison_metrics.")

In [ ]:
# Create a time series graph of daily returns for selected stocks

# Select a few stocks to visualize
selected_stocks = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS']

if all(stock in returns.columns for stock in selected_stocks):
    plt.figure(figsize=(14, 7))
    for stock in selected_stocks:
        returns[stock].plot(label=stock, alpha=0.7)

    plt.title('Daily Returns Time Series for Selected Stocks')
    plt.xlabel('Date')
    plt.ylabel('Daily Return')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("One or more selected stocks not found in the returns data.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calculate annual Sharpe Ratio for each year from 2021 to 2024

annual_sharpe_ratios = {}

for year in range(2021, 2025):
    year_str = str(year)
    returns_year = returns.loc[year_str]

    if not returns_year.empty:
        mean_returns_year = returns_year.mean()
        cov_matrix_year = returns_year.cov()

        # Assuming an equally-weighted portfolio for simplicity for annual Sharpe calculation
        # You could also optimize for each year, but that's more complex and might overfit
        n_assets = returns_year.shape[1]
        equal_weights = np.repeat(1.0/n_assets, n_assets)

        # Calculate annual performance metrics
        # Need to annualize daily metrics: multiply mean by 252, std dev by sqrt(252)
        annual_mean_return = mean_returns_year.mean() * 252
        annual_volatility = returns_year.std().mean() * np.sqrt(252) # Using mean of daily std dev for simplicity

        # Calculate Sharpe Ratio (assuming risk-free rate is 0)
        annual_sharpe = annual_mean_return / (annual_volatility + 1e-9) # Add epsilon to avoid division by zero

        annual_sharpe_ratios[year_str] = annual_sharpe
    else:
        print(f"No returns data for year {year}")


# Visualize annual Sharpe Ratios
if annual_sharpe_ratios:
    years = list(annual_sharpe_ratios.keys())
    sharpes = list(annual_sharpe_ratios.values())

    plt.figure(figsize=(8, 5))
    bars = plt.bar(years, sharpes, color='skyblue')
    plt.ylabel('Sharpe Ratio')
    plt.title('Annual Sharpe Ratio (2021-2024)')
    plt.grid(axis='y')

    # Add the values on top of the bars
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), va='bottom', ha='center')

    plt.show()
else:
    print("No annual Sharpe Ratios calculated for visualization.")

In [ ]:
# Calculate Cumulative Return for CAPM-based portfolio from 2021 to 2025

# Ensure CAPM results, market returns, and risk-free rate are available
if 'capm_df' in locals() and 'market_returns' in locals() and 'rf' in locals():
    # Select the data for the specified period
    returns_2021_2025 = returns.loc['2021':'2025']
    market_returns_2021_2025 = market_returns.loc['2021':'2025']

    if not returns_2021_2025.empty and not market_returns_2021_2025.empty:
        # Calculate expected returns for each stock using CAPM for the period
        # E(Ri) = Rf + Beta_i * (E(Rm) - Rf)
        # We can use the average market return over the period as an estimate for E(Rm)
        average_market_return_2021_2025 = market_returns_2021_2025.mean()

        # Ensure capm_df is aligned with the stocks in returns_2021_2025
        capm_df_aligned = capm_df.reindex(returns_2021_2025.columns)

        # Calculate expected returns based on CAPM
        expected_returns_capm = rf + capm_df_aligned['Beta'] * (average_market_return_2021_2025 - rf)

        # For simplicity, let's assume an equally-weighted portfolio based on CAPM expected returns
        # A more sophisticated approach would be to optimize the portfolio based on these expected returns
        n_assets = returns_2021_2025.shape[1]
        equal_weights = np.repeat(1.0 / n_assets, n_assets)

        # Calculate the daily portfolio return using the actual returns and equal weights
        capm_portfolio_daily_return = returns_2021_2025.dot(equal_weights)

        # Calculate the cumulative return
        capm_cumulative_return = (1 + capm_portfolio_daily_return).cumprod()

        print("\nCAPM-based Equally-Weighted Portfolio Cumulative Return from 2021 to 2025:")
        print(capm_cumulative_return.iloc[-1])  # Print the final cumulative return

        # Optional: Plot the cumulative return
        plt.figure(figsize=(12, 6))
        capm_cumulative_return.plot()
        plt.title('CAPM-based Equally-Weighted Portfolio Cumulative Return (2021-2025)')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.grid(True)
        plt.show()

    else:
        print("No returns data available for the period 2021-2025.")

else:
    print("CAPM results, market returns, or risk-free rate not available.")

In [ ]:
# Calculate Cumulative Return from 2021 to 2025

# Select the returns data from 2021 to 2025
returns_2021_2025 = returns.loc['2021':'2025']

if not returns_2021_2025.empty:
    # Calculate the daily average return of an equally-weighted portfolio
    equal_weight_daily_return = returns_2021_2025.mean(axis=1)

    # Calculate the cumulative return
    cumulative_return = (1 + equal_weight_daily_return).cumprod()

    print("Cumulative Return from 2021 to 2025:")
    print(cumulative_return.iloc[-1]) # Print the final cumulative return

    # Optional: Plot the cumulative return
    plt.figure(figsize=(12, 6))
    cumulative_return.plot()
    plt.title('Equally-Weighted Portfolio Cumulative Return (2021-2025)')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.grid(True)
    plt.show()

else:
    print("No returns data available for the period 2021-2025.")

In [ ]:
# Calculate Cumulative Return for Black-Litterman portfolio from 2021 to 2025

# Ensure Black-Litterman weights and returns data are available
if 'weights_bl' in locals() and 'returns' in locals():
    # Select the returns data for the specified period
    returns_2021_2025 = returns.loc['2021':'2025']

    if not returns_2021_2025.empty:
        # Ensure the order of weights matches the columns in returns_2021_2025
        # Assuming nifty50_tickers list was used to generate the returns columns
        # and weights_bl corresponds to this order.
        # If not, you might need to reindex weights_bl based on returns_2021_2025.columns
        try:
            # Calculate the daily portfolio return using the actual returns and BL weights
            bl_portfolio_daily_return = returns_2021_2025.dot(weights_bl)

            # Calculate the cumulative return
            bl_cumulative_return = (1 + bl_portfolio_daily_return).cumprod()

            print("\nBlack-Litterman Portfolio Cumulative Return from 2021 to 2025:")
            print(bl_cumulative_return.iloc[-1])  # Print the final cumulative return

            # Optional: Plot the cumulative return
            plt.figure(figsize=(12, 6))
            bl_cumulative_return.plot()
            plt.title('Black-Litterman Portfolio Cumulative Return (2021-2025)')
            plt.xlabel('Date')
            plt.ylabel('Cumulative Return')
            plt.grid(True)
            plt.show()

        except ValueError as e:
             print(f"Error calculating Black-Litterman portfolio return. Ensure weights align with returns data: {e}")
    else:
        print("No returns data available for the period 2021-2025.")

else:
    print("Black-Litterman weights or returns data not available.")

In [ ]:
# Plot all three cumulative returns together

# Ensure cumulative return dataframes are available
if 'capm_cumulative_return' in locals() and 'bl_cumulative_return' in locals() and 'cumulative_return' in locals():

    plt.figure(figsize=(12, 6))

    # Plot each cumulative return series
    capm_cumulative_return.plot(label='CAPM-based Portfolio', alpha=0.7)
    bl_cumulative_return.plot(label='Black-Litterman Portfolio', alpha=0.7)
    cumulative_return.plot(label='Equal-Weight Portfolio', alpha=0.7)


    plt.title('Cumulative Return Comparison (2021-2025)')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("Cumulative return data for all models not available for plotting.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cvxpy as cp

# Assuming 'returns', 'mean_returns', and 'Sigma' are available from previous cells

if 'mean_returns' in locals() and 'Sigma' in locals() and not mean_returns.empty and not Sigma.empty:
    n_assets = len(mean_returns)
    returns_np = mean_returns.values
    Sigma_np = Sigma.values

    # Ensure Sigma is positive semi-definite
    def _nearest_psd(A, eps=1e-10):
        B = (A + A.T) / 2
        _, s, V = np.linalg.svd(B)
        s = np.maximum(s, eps)
        return V @ np.diag(s) @ V.T

    Sigma_psd = _nearest_psd(Sigma_np)

    # Define the number of points for the efficient frontier
    n_portfolios = 100
    # Use daily mean returns for the target returns range
    target_returns_daily = np.linspace(mean_returns.min(), mean_returns.max(), n_portfolios)

    portfolio_risks_daily = []
    portfolio_returns_daily = []

    for target_return_daily in target_returns_daily:
        weights = cp.Variable(n_assets)
        risk = cp.quad_form(weights, Sigma_psd)
        constraints = [
            cp.sum(weights) == 1,
            weights >= 0,
            returns_np @ weights >= target_return_daily # Use daily target return
        ]
        prob = cp.Problem(cp.Minimize(risk), constraints)
        try:
            prob.solve(solver=cp.SCS, verbose=False)
            if prob.status in ("optimal", "optimal_inaccurate"):
                portfolio_risks_daily.append(np.sqrt(prob.value)) # Daily risk (standard deviation)
                portfolio_returns_daily.append(target_return_daily)  # Daily return
        except cp.error.SolverError:
            # Handle cases where the solver fails for a given target return
            print(f"Solver failed for target return: {target_return_daily:.6f}")
            continue


    # Plot the efficient frontier (daily)
    plt.figure(figsize=(10, 6))
    plt.scatter(portfolio_risks_daily, portfolio_returns_daily, marker='o', s=10)
    plt.xlabel('Daily Volatility (Risk)')
    plt.ylabel('Daily Return')
    plt.title('Efficient Frontier (Daily)')
    plt.grid(True)
    plt.show()

else:
    print("Mean returns or covariance matrix not available for plotting the efficient frontier.")